In [ ]:
import requests
import bs4
import re
import urllib
from pathlib import Path
from IPython.display import Image, display
import random
import pandas as pd

In [ ]:
base_path = Path.cwd()
source_data_dir = Path(base_path.parent,"MPD-Cleaner")
data_dir = base_path
arrests = pd.read_csv(Path(source_data_dir,"cleaned_arrests.csv"))


In [ ]:
def get_captcha():
    HOME_PAGE = 'https://eaccess.dccourts.gov/eaccess/home.page'
    session = requests.session()
    r = session.get(HOME_PAGE)

    assert r.ok
    soup = bs4.BeautifulSoup(r.content, 'html.parser')
    url = re.match(r'^.*url=(.*)(?= .*)?$', soup.find('meta').attrs['content']).groups()[0]

    r = session.get(urllib.parse.urljoin(HOME_PAGE, url))
    soup = bs4.BeautifulSoup(r.content, 'html.parser')

    # Get the captcha
    the_form = soup.findAll('form')[0]
    the_form.findAll('input')
    url = the_form.findAll('img', {'class': 'captchaImg'})[0].attrs['src']
    img_url = urllib.parse.urljoin(r.url, url)
    img_r = session.get(img_url)
    return session, r, soup, url, img_r, the_form

session, r, soup, url, img_r, the_form = get_captcha()
assert img_r.ok
display(Image(img_r.content))
captcha_content = input('captcha text: ')

In [ ]:
appendix = re.match(r"^.*'(\?x=.*?)'.*$", the_form.find('a', attrs={'name': 'linkFrag:beginButton'}).attrs['onclick']).groups()[0]
action_url = urllib.parse.urljoin(r.url, appendix)
the_form.attrs['method']

new_r = session.post(action_url, data={
    'captchaPanel:challengePassword': captcha_content,
    'idb_hf_0': None,
    'linkFrag:beginButton': 1 })

assert new_r.ok


In [ ]:
search_by_name_base = "https://eaccess.dccourts.gov/eaccess/search.page"
new_soup = bs4.BeautifulSoup(new_r.content, 'html.parser')
ahref = new_soup.findAll('a')[15]
link_string =  ahref.attrs['href']
r = session.get(search_by_name_base+link_string)
assert r.ok
soup = bs4.BeautifulSoup(r.content, 'html.parser')


In [ ]:
by_name_form = soup.find('form')
by_name_form.findAll('input')


In [ ]:
appendix = re.match(r"^.*'(\?x=.*?)'.*$", the_form.find('a', attrs={'name': 'linkFrag:beginButton'}).attrs['onclick']).groups()[0]
action_url = urllib.parse.urljoin(r.url, appendix)


In [ ]:
arrest = arrests.loc[len(arrests)-1]
first_name = arrest.consumer_first_name
last_name = arrest.consumer_last_name
dob_start = arrest.consumer_dob_start
dob_end = arrest.consumer_dob_end
first_name, last_name, dob_start, dob_end

In [ ]:
name_dob_r = session.post(action_url, data={
    'lastName': last_name,
    'first_name': first_name,
    'dobDateRange:dateInputBegin': dob_start,
    'dobDateRange:dateInputEnd': dob_end,
    'linkFrag:beginButton': 1 })


In [ ]:
assert r.ok
soup = bs4.BeautifulSoup(r.content, 'html.parser')


In [ ]:
soup